In [20]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import time

In [21]:
from sequence.kernel.timeline import Timeline
from sequence.topology.node import QuantumRouter, MiddleNode
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel
from sequence.protocols.management.rule_manager import Rule
from sequence.protocols.entanglement.generation import EntanglementGenerationA

In [22]:
def eg_rule_condition(memory_info, manager):
    if memory_info.state == "RAW":
        return [memory_info]
    else:
        return []

def eg_rule_action1(memories):
    def eg_req_func(protocol):
        return isinstance(protocol, EntanglementGenerationA)

    memory = memories[0]
    protocol = EntanglementGenerationA(None, "EGA." + memory.name, "m1", "r2", memory)
    protocol.primary = True
    return [protocol, "r2", eg_req_func]

def eg_rule_action2(memories):
    memory = memories[0]
    protocol = EntanglementGenerationA(None, "EGA." + memory.name, "m1", "r1", memory)
    return [protocol, None, None]

In [23]:
def test(sim_time, cc_delay, qc_atten, qc_dist):
    """
    sim_time: duration of simulation time (ms)
    cc_delay: delay on classical channels (ns)
    qc_atten: attenuation on quantum channels (db/m)
    qc_dist: distance of quantum channels (km)
    """
    cc_delay *= 1e6
    qc_dist *= 1e3
    tl = Timeline(sim_time * 1e9)
    
    r1 = QuantumRouter("r1", tl)
    r2 = QuantumRouter("r2", tl)
    m1 = MiddleNode("m1", tl, ["r1", "r2"])
    
    cc1 = ClassicalChannel("cc_r1_m1", tl, 2e-4, 1e3, delay=cc_delay)
    cc1.set_ends(r1, m1)
    cc2 = ClassicalChannel("cc_r2_m1", tl, 2e-4, 1e3, delay=cc_delay)
    cc2.set_ends(r2, m1)
    cc3 = ClassicalChannel("cc_r1_r2", tl, 2e-4, 1e3, delay=cc_delay)
    cc3.set_ends(r1, r2)
    
    qc1 = QuantumChannel("qc_r1_m1", tl, qc_atten, qc_dist)
    qc1.set_ends(r1, m1)
    qc2 = QuantumChannel("qc_r2_m1", tl, qc_atten, qc_dist)
    qc2.set_ends(r2, m1)
    
    tl.init()
    rule1 = Rule(10, eg_rule_action1, eg_rule_condition)
    r1.resource_manager.load(rule1)
    rule2 = Rule(10, eg_rule_action2, eg_rule_condition)
    r2.resource_manager.load(rule2)
    
    tick = time.time()
    tl.run()
    print("execution time %.2f sec" % (time.time() - tick))
    
    data = []
    for info in r1.resource_manager.memory_manager:
        data.append(info.entangle_time / 1e12)
    data.sort()
    plt.plot(data, range(1, len(data) + 1), marker="o")
    plt.xlabel("Simulation Time (s)")
    plt.ylabel("Aggregated Number of Entangled Memory")
    plt.show()
    

In [24]:
interactive_plot = interact(test, sim_time=(100, 1000, 100), cc_delay=(100, 10000, 100), qc_atten=[1e-5, 2e-5, 3e-5], qc_dist=(1, 10, 1))
interactive_plot

interactive(children=(IntSlider(value=500, description='sim_time', max=1000, min=100, step=100), IntSlider(val…

<function __main__.test(sim_time, cc_delay, qc_atten, qc_dist)>